In [45]:
import os
import torch
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

PyTorch version: 1.13.1
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [46]:
folder = "Data/Premier"
años = ["19-20", "20-21", "21-22", "22-23", "23-24"]
torneos = []

for año in años:
    match_file = f"stats_{año}.csv"
    path_matches = os.path.join(folder, match_file)
    matches = pd.read_csv(path_matches)
    torneos.append(matches)
    print(matches.head())

results_last_season = pd.read_csv("Data/premier-league-tables.csv")

#Solo nos quedamos con las columnas que nos interesan
results_last_season = results_last_season[["Season_End_Year", "Team", "Rk"]]
print(results_last_season.head())

   number_of_match          team_home               team_away  home_score  \
0                2        Bournemouth        Sheffield United         1.0   
1                9  Manchester United                 Chelsea         4.0   
2                5            Watford  Brighton & Hove Albion         0.0   
3               30        Southampton       Manchester United         1.0   
4               20        Aston Villa                 Everton         2.0   

   away_score  posession_home  posession_away  total_shots_home  \
0         1.0            52.0            48.0              13.0   
1         0.0            46.0            54.0              11.0   
2         3.0            48.0            52.0              11.0   
3         1.0            41.0            59.0              10.0   
4         0.0            35.0            65.0               7.0   

   total_shots_away  shots_on_target_home  ...  duels_won_home  \
0               8.0                   3.0  ...            56.0   
1 

In [47]:
for torneo in torneos:
    torneo.dropna(subset=['team_home', 'team_away'], inplace=True)

columns_with_null = []

for torneo in torneos:
    null_columns = torneo.columns[torneo.isna().any()].tolist()
    columns_with_null.append(null_columns)

columns_with_null

unique_columns = list(set(itertools.chain.from_iterable(columns_with_null)))
unique_columns

#Borrar para cada torneo las columnas en unique_columns
for torneo in torneos:
    torneo.drop(columns=unique_columns, inplace=True)

torneos2 = []

for torneo in torneos:
    torneo.sort_values(by='number_of_match', inplace=True)
    torneo.rename(columns={'home_score': 'score_home', 'away_score': 'score_away'}, inplace=True)

    ##OJOO arreglar tema de equipos
    team_counts = pd.concat([torneo['team_home'], torneo['team_away']]).value_counts()
    teams_to_remove = team_counts[team_counts < 5].index

    torneo = torneo[~(torneo['team_home'].isin(teams_to_remove) | torneo['team_away'].isin(teams_to_remove))]
    torneos2.append(torneo)

torneos = torneos2
torneos[0].head()

#Cantidad de datos en cada torneo
for torneo in torneos:
    print(torneo.shape)

(378, 47)
(380, 47)
(374, 47)
(380, 47)
(132, 47)


In [48]:
for torneo, i in zip(torneos, range(len(torneos))):

    torneo.loc[:, "team_home"] = torneo["team_home"].apply(lambda x: x.replace("AFC ", ""))
    torneo.loc[:, "team_away"] = torneo["team_away"].apply(lambda x: x.replace("AFC ", ""))

    torneo.loc[:, "team_home"] = torneo["team_home"].apply(lambda x: x.replace("West Ham United", "West Ham"))
    torneo.loc[:, "team_away"] = torneo["team_away"].apply(lambda x: x.replace("West Ham United", "West Ham"))

    torneo.loc[:, "team_home"] = torneo["team_home"].apply(lambda x: x.replace(" Hotspur", ""))
    torneo.loc[:, "team_away"] = torneo["team_away"].apply(lambda x: x.replace(" Hotspur", ""))

    torneo.loc[:, "team_home"] = torneo["team_home"].apply(lambda x: x.replace("Brighton & Hove Albion", "Brighton"))
    torneo.loc[:, "team_away"] = torneo["team_away"].apply(lambda x: x.replace("Brighton & Hove Albion", "Brighton"))

    equipos = torneo["team_home"].unique()
    equipos2 = torneo["team_away"].unique()

    print(len(equipos), len(equipos2))
    print(equipos)



20 20
['Liverpool' 'West Ham' 'Bournemouth' 'Burnley' 'Crystal Palace' 'Watford'
 'Tottenham' 'Newcastle United' 'Manchester United' 'Arsenal'
 'Aston Villa' 'Brighton' 'Everton' 'Norwich City' 'Southampton'
 'Manchester City' 'Sheffield United' 'Chelsea' 'Wolverhampton'
 'Leicester City']
20 20
['Fulham' 'Crystal Palace' 'Liverpool' 'West Ham' 'West Bromwich Albion'
 'Tottenham' 'Sheffield United' 'Brighton' 'Burnley' 'Manchester City'
 'Everton' 'Leeds United' 'Manchester United' 'Arsenal' 'Southampton'
 'Newcastle United' 'Chelsea' 'Leicester City' 'Aston Villa'
 'Wolverhampton']
20 20
['Brentford' 'Manchester United' 'Burnley' 'Chelsea' 'Everton'
 'Leicester City' 'Watford' 'Norwich City' 'Newcastle United' 'Tottenham'
 'Liverpool' 'Aston Villa' 'Crystal Palace' 'Leeds United'
 'Manchester City' 'Brighton' 'Southampton' 'Wolverhampton' 'Arsenal'
 'West Ham']
20 20
['Crystal Palace' 'Fulham' 'Bournemouth' 'Leeds United' 'Newcastle United'
 'Tottenham' 'Everton' 'Leicester City' 'Man

In [49]:
#Cambiar nombre de teams:
#Manchester Utd -> Manchester United
results_last_season["Team"] = results_last_season["Team"].replace("Manchester Utd", "Manchester United")
#Newcastle Utd -> Newcastle United
results_last_season["Team"] = results_last_season["Team"].replace("Newcastle Utd", "Newcastle United")
#Nott'm Forest -> Nottingham Forest
results_last_season["Team"] = results_last_season["Team"].replace("Nott'm Forest", "Nottingham Forest")
#Wolves -> Wolverhampton
results_last_season["Team"] = results_last_season["Team"].replace("Wolves", "Wolverhampton")

#desde 19-20 hasta 23-24
ini = 2019
for i in range(len(torneos)):
    df = torneos[i]

    #cantidad de datos

    print(f"cant.datos:\n {df.shape}")

    results = results_last_season[results_last_season["Season_End_Year"] == ini]

    #columnas de results_2019
    print(results.columns)

    #Los equipos que esten en la tabla de la temporada pero no en el torneo, se les asigna la posición 18
    teams_in_tournament = df["team_home"].unique()
    print("\nTeams in tournament Home")
    print(teams_in_tournament)

    teams_in_tournament2 = df["team_away"].unique()
    print("\nTeams in tournament Away")
    print(teams_in_tournament2)

    teams_in_table = results["Team"].unique()
    print("\nTeams in table")
    print(teams_in_table)

    teams_not_in_tournament = np.setdiff1d(teams_in_tournament, teams_in_table)
    print("\nTeams not in tournament")
    print(teams_not_in_tournament)

    for team in teams_not_in_tournament:
        results = pd.concat([results, pd.DataFrame({"Team": [team], "Rk": [18], "Season_End_Year": [ini]})])
        print(results)
        #Unir los dos dataframes con team_home y team_away
    df = df.merge(results, left_on="team_home", right_on="Team")
    #Botar la columna Team y Season_End_Year
    df = df.drop(columns=["Team", "Season_End_Year"])
    #Renombrar la columna Rk a last_season_position_home
    df = df.rename(columns={"Rk": "last_season_position_home"})
    #Unir con team_away
    df = df.merge(results, left_on="team_away", right_on="Team")
    #Botar la columna Team y Season_End_Year
    df = df.drop(columns=["Team", "Season_End_Year"])
    #Renombrar la columna Rk a last_season_position_away
    df = df.rename(columns={"Rk": "last_season_position_away"})
    print(df.head())

    #exportar a stat
    df.to_csv(f"Data/stats_{str(ini)[2:]}-{str(ini+1)[2:]}.csv", index=False)
    print(f"Data/stats_{str(ini)[2:]}-{str(ini+1)[2:]}.csv")
    ini += 1

cant.datos:
 (378, 47)
Index(['Season_End_Year', 'Team', 'Rk'], dtype='object')

Teams in tournament Home
['Liverpool' 'West Ham' 'Bournemouth' 'Burnley' 'Crystal Palace' 'Watford'
 'Tottenham' 'Newcastle United' 'Manchester United' 'Arsenal'
 'Aston Villa' 'Brighton' 'Everton' 'Norwich City' 'Southampton'
 'Manchester City' 'Sheffield United' 'Chelsea' 'Wolverhampton'
 'Leicester City']

Teams in tournament Away
['Norwich City' 'Manchester City' 'Sheffield United' 'Southampton'
 'Everton' 'Brighton' 'Aston Villa' 'Arsenal' 'Chelsea' 'Burnley'
 'Bournemouth' 'West Ham' 'Watford' 'Newcastle United' 'Liverpool'
 'Tottenham' 'Crystal Palace' 'Leicester City' 'Manchester United'
 'Wolverhampton']

Teams in table
['Arsenal' 'Bournemouth' 'Brighton' 'Burnley' 'Cardiff City' 'Chelsea'
 'Crystal Palace' 'Everton' 'Fulham' 'Huddersfield' 'Leicester City'
 'Liverpool' 'Manchester City' 'Manchester United' 'Newcastle United'
 'Southampton' 'Tottenham' 'Watford' 'West Ham' 'Wolverhampton']

Teams 